In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data_dir = f"datasets/abalone"
results_dir_relu = "results/regression/single_layer/relu/abalone"
results_dir_tanh = "results/regression/single_layer/tanh/abalone"
#model_names_relu = ["Dirichlet Student T"]
model_names_relu = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh"]


full_config_path = "abalone_N3341_p8"
relu_fit = get_model_fits(
    config=full_config_path,
    results_dir=results_dir_relu,
    models=model_names_relu,
    include_prior=False,
)

tanh_fit = get_model_fits(
    config=full_config_path,
    results_dir=results_dir_tanh,
    models=model_names_tanh,
    include_prior=False,
)

    


In [ ]:
from sklearn.metrics import mean_squared_error
from properscoring import crps_ensemble
import numpy as np
import pandas as pd

# Load data
from utils.generate_data import load_abalone_regression_data
X_train, X_test, y_train, y_test = load_abalone_regression_data(standardized=False, frac=1.0)

results = []

for model_name, model_entry in tanh_fit.items():
    posterior = model_entry["posterior"]

    # Get posterior predictive samples for the test set (shape: [n_draws, n_test])
    y_pred_samples = posterior.stan_variable("output_test").squeeze(-1)  # shape: (n_draws, n_test)

    # Compute posterior mean predictions
    y_pred_mean = np.mean(y_pred_samples, axis=0)
    
    # Compute RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_mean))

    # Compute CRPS
    #crps = np.mean(crps_ensemble(y_test, y_pred_samples.T))  # crps_ensemble wants shape (n_test, n_draws)
    
    #max_draws = 500
    #idx = np.random.choice(y_pred_samples.shape[0], max_draws, replace=False)
    #y_pred_samples_sub = y_pred_samples[idx, :]
    #crps = np.mean(crps_ensemble(y_test, y_pred_samples.T))


    results.append({
        "Model": model_name,
        "RMSE": rmse,
    #    "CRPS": crps
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)
print(results_df)


In [ ]:
from properscoring import crps_ensemble
import numpy as np
import pandas as pd
from utils.generate_data import load_abalone_regression_data
X_train, X_test, y_train, y_test = load_abalone_regression_data(standardized=False, frac=1.0)
y_preds = relu_fit['Gaussian']['posterior'].stan_variable("output_test").squeeze(-1)

In [ ]:
max_draws = 10
idx = np.random.choice(y_preds.shape[0], max_draws, replace=False)
y_pred_samples_sub = y_preds[idx, :]
crps = np.mean(crps_ensemble(y_test, y_preds.T))

In [ ]:
crps

In [ ]:
latex_table = results_df.to_latex(index=False, float_format="%.4f", column_format="lcc", caption="RMSE and CRPS per model.", label="tab:rmse_crps")
print(latex_table)

In [4]:
from utils.generate_data import load_abalone_regression_data
def compute_sparse_rmse_results_abalone(models, all_fits, forward_pass,
                         sparsity=0.0, prune_fn=None):
    results = []
    posterior_means = []
    for model in models:
        try:
            fit = all_fits[model]['posterior']
            W1_samples = fit.stan_variable("W_1")           # (S, P, H)
            W2_samples = fit.stan_variable("W_L")           # (S, H, O)
            b1_samples = fit.stan_variable("hidden_bias")   # (S, O, H)
            b2_samples = fit.stan_variable("output_bias")   # (S, O)
        except KeyError:
            print(f"[SKIP] Model or posterior not found:")
            continue

        S = W1_samples.shape[0]
        rmses = np.zeros(S)
        #print(y_test.shape)
        _, X_test, _, y_test = load_abalone_regression_data(standardized=False, frac=1.0)
        y_hats = np.zeros((S, y_test.shape[0]))

        for i in range(S):
            W1 = W1_samples[i]
            W2 = W2_samples[i]

            # Apply pruning mask if requested
            if prune_fn is not None and sparsity > 0.0:
                masks = prune_fn([W1, W2], sparsity)
                W1 = W1 * masks[0]
                #W2 = W2 * masks[1]

            y_hat = forward_pass(X_test, W1, b1_samples[i][0], W2, b2_samples[i])
            y_hats[i] = y_hat.squeeze()  # Store the prediction for each sample
            rmses[i] = np.sqrt(np.mean((y_hat.squeeze() - y_test)**2))
            
        posterior_mean = np.mean(y_hats, axis=0)
        posterior_mean_rmse = np.sqrt(np.mean((posterior_mean - y_test.squeeze())**2))

        posterior_means.append({
            'model': model,
            'sparsity': sparsity,
            'posterior_mean_rmse': posterior_mean_rmse
        })

        for i in range(S):
            results.append({
                'model': model,
                'sparsity': sparsity,
                'rmse': rmses[i]
            })

    df_rmse = pd.DataFrame(results)
    df_posterior_rmse = pd.DataFrame(posterior_means)

    return df_rmse, df_posterior_rmse


In [5]:
from utils.sparsity import forward_pass_relu, forward_pass_tanh, local_prune_weights

sparsity_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

df_rmse_relu, df_posterior_rmse_relu = {}, {}
df_rmse_tanh, df_posterior_rmse_tanh = {}, {}

for sparsity in sparsity_levels:
    df_rmse_relu[sparsity], df_posterior_rmse_relu[sparsity] = compute_sparse_rmse_results_abalone(
        models = model_names_relu,
        all_fits = relu_fit, 
        forward_pass = forward_pass_relu,
        sparsity=sparsity, 
        prune_fn=local_prune_weights
    )

    df_rmse_tanh[sparsity], df_posterior_rmse_tanh[sparsity] = compute_sparse_rmse_results_abalone(
        models = model_names_tanh,
        all_fits = tanh_fit, 
        forward_pass = forward_pass_tanh,
        sparsity=sparsity, 
        prune_fn=local_prune_weights
    )


In [ ]:
# Combine
df_rmse_full_relu = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_rmse_relu.items()],
    ignore_index=True
)

df_rmse_full_tanh = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_rmse_tanh.items()],
    ignore_index=True
)

# Plot (simplified version)
import matplotlib.pyplot as plt
import seaborn as sns
custom_palette = {
    "Gaussian": "C0",
    "Regularized Horseshoe": "C1",
    "Dirichlet Horseshoe": "C2",
    "Dirichlet Student T": "C3",
}
# Clean names
df_rmse_full_relu["model"] = df_rmse_full_relu["model"].str.replace(" tanh", "", regex=False)
df_rmse_full_tanh["model"] = df_rmse_full_tanh["model"].str.replace(" tanh", "", regex=False)

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharex=True, sharey=True)
activation_data = [("ReLU", df_rmse_full_relu), ("tanh", df_rmse_full_tanh)]

for ax, (name, df) in zip(axes, activation_data):
    sns.lineplot(
        data=df,
        x='sparsity', y='rmse',
        hue='model', marker='o', errorbar=None, ax=ax,
        palette=custom_palette,
    )
    ax.set_title(name)
    ax.set_xlabel("Sparsity level")
    ax.set_ylabel("RMSE")
    ax.grid(True)
    ax.legend(title="Model")

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Combine only ReLU
df_rmse_full_relu = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_rmse_relu.items()],
    ignore_index=True
)

# Clean model names
df_rmse_full_relu["model"] = df_rmse_full_relu["model"].str.replace(" tanh", "", regex=False)

# Custom colors
custom_palette = {
    "Gaussian": "C0",
    "Regularized Horseshoe": "C1",
    "Dirichlet Horseshoe": "C2",
    "Dirichlet Student T": "C3",
}

# Plot
plt.figure(figsize=(7, 5))
sns.lineplot(
    data=df_rmse_full_relu,
    x='sparsity', y='rmse',
    hue='model', marker='o', errorbar=None,
    palette=custom_palette,
)
plt.title("ReLU")
plt.xlabel("Sparsity level")
plt.ylabel("RMSE")
plt.grid(True)
plt.legend(title="Model")
plt.tight_layout()
plt.show()


In [8]:
from utils.generate_data import load_abalone_regression_data
X_train, X_test, y_train, y_test = load_abalone_regression_data(standardized=False, frac=1.0)


In [9]:
model_names = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]

model_preds = {}
for model_name in model_names:
    preds = relu_fit[model_name]['posterior'].stan_variable("output_test_rng")
    model_preds[model_name] = {
        "mean": np.mean(preds, axis=0).squeeze(-1),
        "std": np.std(preds, axis=0).squeeze(-1),
        "lower_95": np.percentile(preds, 2.5, axis=0).squeeze(-1),
        "upper_95": np.percentile(preds, 97.5, axis=0).squeeze(-1),
    }


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(14, 12))
axs = axs.flatten()

# Get global limits for consistent scaling
all_preds = np.concatenate([model_preds[m]["mean"] for m in model_names])
y_min = min(y_test.min(), all_preds.min())
y_max = max(y_test.max(), all_preds.max())

for i, model_name in enumerate(model_names):
    ax = axs[i]
    sc = ax.scatter(
        y_test, model_preds[model_name]["mean"],
        c=model_preds[model_name]["std"], cmap='viridis', alpha=0.7
    )
    ax.plot([y_min, y_max], [y_min, y_max], 'k--', lw=1)
    ax.set_title(f"{model_name}")
    ax.set_xlabel("True y")
    ax.set_ylabel("Predicted mean")
    ax.grid(True)
    ax.set_xlim(y_min, y_max)
    ax.set_ylim(y_min, y_max)

cbar = fig.colorbar(sc, ax=axs, orientation='vertical', fraction=0.02, pad=0.04)
cbar.set_label("Predictive Std Dev")
plt.suptitle("Regression Predictions with Uncertainty", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
coverage = {}
for model_name in model_names:
    lower = model_preds[model_name]["lower_95"]
    upper = model_preds[model_name]["upper_95"]
    inside = (y_test >= lower) & (y_test <= upper)
    coverage[model_name] = np.mean(inside)

# Bar plot
plt.figure(figsize=(7, 5))
plt.bar(coverage.keys(), coverage.values())
plt.axhline(0.95, color='red', linestyle='--', label='Ideal Coverage (95%)')
plt.ylabel("Proportion Covered")
plt.title("Coverage of 95% Predictive Intervals")
plt.ylim(0, 1)
plt.grid(True, axis='y')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
for model_name in model_names:
    errors = np.abs(model_preds[model_name]["mean"] - y_test)
    stds = model_preds[model_name]["std"]
    plt.scatter(stds, errors, label=model_name, alpha=0.5)

plt.xlabel("Predictive Std Dev")
plt.ylabel("Absolute Error")
plt.title("Error vs. Uncertainty")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
for model_name in model_names:
    pred = model_preds[model_name]["mean"]
    mae = np.mean(np.abs(pred - y_test))
    rmse = np.sqrt(np.mean((pred - y_test) ** 2))
    width = np.mean(model_preds[model_name]["upper_95"] - model_preds[model_name]["lower_95"])
    print(f"{model_name}: MAE = {mae:.3f}, RMSE = {rmse:.3f}, Avg Interval Width = {width:.3f}")


In [14]:
model_names = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh"]

model_preds = {}
for model_name in model_names:
    preds = tanh_fit[model_name]['posterior'].stan_variable("output_test_rng")
    model_preds[model_name] = {
        "mean": np.mean(preds, axis=0).squeeze(-1),
        "std": np.std(preds, axis=0).squeeze(-1),
        "lower_95": np.percentile(preds, 2.5, axis=0).squeeze(-1),
        "upper_95": np.percentile(preds, 97.5, axis=0).squeeze(-1),
    }


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(14, 12))
axs = axs.flatten()

for i, model_name in enumerate(model_names):
    ax = axs[i]
    sc = ax.scatter(y_test, model_preds[model_name]["mean"],
                    c=model_preds[model_name]["std"], cmap='viridis', alpha=0.7)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=1)
    ax.set_title(f"{model_name}")
    ax.set_xlabel("True y")
    ax.set_ylabel("Predicted mean")
    ax.grid(True)
    
cbar = fig.colorbar(sc, ax=axs, orientation='vertical', fraction=0.02, pad=0.04)
cbar.set_label("Predictive Std Dev")
plt.suptitle("Regression Predictions with Uncertainty", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
coverage = {}
for model_name in model_names:
    lower = model_preds[model_name]["lower_95"]
    upper = model_preds[model_name]["upper_95"]
    inside = (y_test >= lower) & (y_test <= upper)
    coverage[model_name] = np.mean(inside)

# Bar plot
plt.figure(figsize=(7, 5))
plt.bar(coverage.keys(), coverage.values())
plt.axhline(0.95, color='red', linestyle='--', label='Ideal Coverage (95%)')
plt.ylabel("Proportion Covered")
plt.title("Coverage of 95% Predictive Intervals")
plt.ylim(0, 1)
plt.grid(True, axis='y')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
for model_name in model_names:
    errors = np.abs(model_preds[model_name]["mean"] - y_test)
    stds = model_preds[model_name]["std"]
    plt.scatter(stds, errors, label=model_name, alpha=0.5)

plt.xlabel("Predictive Std Dev")
plt.ylabel("Absolute Error")
plt.title("Error vs. Uncertainty")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
for model_name in model_names:
    pred = model_preds[model_name]["mean"]
    mae = np.mean(np.abs(pred - y_test))
    rmse = np.sqrt(np.mean((pred - y_test) ** 2))
    width = np.mean(model_preds[model_name]["upper_95"] - model_preds[model_name]["lower_95"])
    print(f"{model_name}: MAE = {mae:.3f}, RMSE = {rmse:.3f}, Avg Interval Width = {width:.3f}")


## ONLY FULL REGULARIZATION

In [ ]:
data_dir = f"datasets/abalone"
results_dir_relu = "results/regression/single_layer/relu/abalone/full_regularization"
results_dir_tanh = "results/regression/single_layer/tanh/abalone/full_regularization"
#model_names_relu = ["Dirichlet Student T"]
model_names_relu = ["Dirichlet Student T"]
model_names_tanh = ["Dirichlet Student T tanh"]


full_config_path = "abalone_N3341_p8"
relu_fit = get_model_fits(
    config=full_config_path,
    results_dir=results_dir_relu,
    models=model_names_relu,
    include_prior=False,
)

tanh_fit = get_model_fits(
    config=full_config_path,
    results_dir=results_dir_tanh,
    models=model_names_tanh,
    include_prior=False,
)

    


In [ ]:
from sklearn.metrics import mean_squared_error
from properscoring import crps_ensemble
import numpy as np
import pandas as pd

# Load data
from utils.generate_data import load_abalone_regression_data
X_train, X_test, y_train, y_test = load_abalone_regression_data(standardized=False, frac=1.0)

results = []

for model_name, model_entry in tanh_fit.items():
    posterior = model_entry["posterior"]

    # Get posterior predictive samples for the test set (shape: [n_draws, n_test])
    y_pred_samples = posterior.stan_variable("output_test").squeeze(-1)  # shape: (n_draws, n_test)

    # Compute posterior mean predictions
    y_pred_mean = np.mean(y_pred_samples, axis=0)
    
    # Compute RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_mean))

    # Compute CRPS
    #crps = np.mean(crps_ensemble(y_test, y_pred_samples.T))  # crps_ensemble wants shape (n_test, n_draws)
    
    #max_draws = 500
    #idx = np.random.choice(y_pred_samples.shape[0], max_draws, replace=False)
    #y_pred_samples_sub = y_pred_samples[idx, :]
    #crps = np.mean(crps_ensemble(y_test, y_pred_samples.T))


    results.append({
        "Model": model_name,
        "RMSE": rmse,
    #    "CRPS": crps
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)
print(results_df)
